In [410]:
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd
from datetime import timedelta
import re

In [411]:

lac_2022_df = pd.read_csv('data/lac_2022_pbp.csv')
lac_2022_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback."
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...


In [412]:
roster_df = pd.read_csv('data/lac_2022_roster.csv')
roster_list=roster_df['roster'].tolist()
print(roster_list)
#roster_df.head()

['Nasir Adderley', 'Keenan Allen', 'Jeremiah Attaochu', 'Zack Bailey', 'Michael Bandy', 'Taylor Bertolet', 'Joey Bosa', 'Bryce Callahan', 'DeAndre Carter', 'Will Clapp', 'Christian Covington', 'Chase Daniel', 'Michael Davis', 'Tyeler Davison', 'Cameron Dicker', 'Austin Ekeler', 'Gerald Everett', 'Breiden Fehoko', 'Matt Feiler', 'Morgan Fox', 'Joe Gaziano', 'Alohi Gilman', 'Jalen Guyton', 'Kemon Hall', 'Josh Harris', 'Justin Herbert', 'Christopher Hinton', 'Dustin Hopkins', 'Zander Horvath', 'J.C. Jackson', 'Brenden Jaimes', 'Derwin James', 'Austin Johnson', 'Zion Johnson', 'Sebastian Joseph-Day', 'Joshua Kelley', 'Raheem Layne', 'Deane Leonard', 'Corey Linsley', 'Khalil Mack', "Tre' McKitty", 'Sony Michel', 'David Moa', 'Jason Moore', 'Kenneth Murray', 'Nick Niemann', 'Storm Norton', 'Amen Ogbongbemiga', 'Otito Ogbonnia', 'Josh Palmer', 'Donald Parham', 'Trey Pipkins', 'Troy Reeder', 'Richard Rodgers', 'Larry Rountree', 'Chris Rumph', 'Jamaree Salyer', 'Asante Samuel Jr.', 'Foster Sare

In [413]:
def possession_check(sentence, roster_list,team_1_abrv,):
    # Split the sentence into words
    words = sentence.split()
    # each detail begins with the name of a player on the 
    name = ' '.join(words[:2])
    # Check if the name is on thhe roster list, if not then it is the other
    if name in roster_list:
        return team_1_abrv
    else:
        return #"OPP"

# Define your list of words to check against
# Apply the function to create a new column called "possession_team"






In [414]:
lac_2022_df['possession_team'] = lac_2022_df['Detail'].apply(lambda x: possession_check(x, roster_list, "LAC"))
lac_2022_df = lac_2022_df[lac_2022_df['possession_team'] == 'LAC']
rows_to_delete = lac_2022_df[lac_2022_df['Detail'].str.contains('punt|kick|goal|kneels', case=False)].index
lac_2022_df.drop(rows_to_delete, inplace=True)
lac_2022_df.reset_index(drop=True, inplace=True)
lac_off_df = lac_2022_df.copy()
lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,LAC


In [415]:
lac_off_df = lac_off_df[(lac_off_df['Quarter'] != "OT")]
lac_off_df = lac_off_df.dropna()
#lac_off_df = lac_off_df.astype({'Quarter': 'int8', 'Down': 'int8','ToGo': 'int8','Down': 'int8','Location':'string','OPP':'string','OPP Score':'int8','Detail':'string'})
#'Time':'string'
#lac_off_df_clean=lac_off_df.copy()
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,LAC


In [416]:
def detect_value_changes(df, column_name):
    change_indices = []
    prev_value = None
    
    for index, row in df.iterrows():
        current_value = row[column_name]
        if prev_value is not None and current_value != prev_value:
            change_indices.append(index)
        prev_value = current_value
    
    return change_indices

# Initialize 'team change' column with zeros
lac_off_df['team change'] = 0

# Update 'team change' with ones where there's a change in 'OPP' column
changes = detect_value_changes(lac_off_df, 'OPP')
for index in changes:
    lac_off_df.loc[index, 'team change'] = 1

lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,LAC,0


In [417]:
import re

# Initialize cumulative rushing yards
cumulative_rush_yards = 0
team_change_index = None  # Initialize variable to store the index where team_change is 1

# Define match words and iterate through the DataFrame to update rush-related columns
match_words = ['middle for', 'tackle for', 'guard for', 'end for']
for index, row in lac_off_df.iterrows():
    detail = row['Detail']
    rush_play = 0
    rush_length = 0
    for word in match_words:
        if word in detail:
            yards_match = re.search(r'for\s+(-?\d+)\s+yards', detail)
            if yards_match:
                rush_play = 1
                rush_length = int(yards_match.group(1))
                break  # Exit loop once a match is found

    # Update rush-related columns
    lac_off_df.at[index, 'rush_play'] = rush_play
    lac_off_df.at[index, 'rush_length'] = rush_length

    # Reset rush yards tally and team_change_index if team change equals 1
    if row['team change'] == 1:
        cumulative_rush_yards = 0
        team_change_index = index

    # Update rush yards tally
    if rush_play == 1:
        cumulative_rush_yards += rush_length
    lac_off_df.at[index, 'rush_yards_tally'] = cumulative_rush_yards

lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change,rush_play,rush_length,rush_yards_tally
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0,1.0,8.0,8.0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0,0.0,0.0,8.0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0,1.0,2.0,10.0
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0,1.0,2.0,12.0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,LAC,0,0.0,0.0,12.0


In [418]:
# Initialize cumulative passing yards
cumulative_pass_yards = 0
team_change_index = None  # Initialize variable to store the index where team_change is 1

# Iterate through the DataFrame to update pass-related columns and pass yards tally
for index, row in lac_off_df.iterrows():
    detail = row['Detail']
    pass_play = 0
    pass_complete = 0
    pass_incomplete = 0
    pass_length = 0
    
    if "pass incomplete" in detail:
        pass_play = 1
        pass_incomplete = 1
    elif "pass complete" in detail:
        pass_play = 1
        pass_complete = 1
        yards_match = re.search(r'for\s+(-?\d+)\s+yards', detail)
        if yards_match:
            pass_length = int(yards_match.group(1))
            cumulative_pass_yards += pass_length  # Update cumulative pass yards tally
    
    # Update pass-related columns
    lac_off_df.at[index, 'pass_play'] = pass_play
    lac_off_df.at[index, 'pass_complete'] = pass_complete
    lac_off_df.at[index, 'pass_incomplete'] = pass_incomplete
    lac_off_df.at[index, 'pass_length'] = pass_length
    
    # Reset pass yards tally and team_change_index if team change equals 1
    if row['team change'] == 1:
        cumulative_pass_yards = 0
        team_change_index = index
    
    # Update pass yards tally
    lac_off_df.at[index, 'pass_yards_tally'] = cumulative_pass_yards

lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change,rush_play,rush_length,rush_yards_tally,pass_play,pass_complete,pass_incomplete,pass_length,pass_yards_tally
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0,1.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0,0.0,0.0,8.0,1.0,1.0,0.0,0.0,0.0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0,1.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0,1.0,2.0,12.0,0.0,0.0,0.0,0.0,0.0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,LAC,0,0.0,0.0,12.0,1.0,1.0,0.0,9.0,9.0


In [419]:
# Set initial value for the first row
lac_off_df.at[lac_off_df.index[0], 'yardage_diff_1_2'] = lac_off_df.at[lac_off_df.index[0], 'ToGo']
lac_off_df.at[lac_off_df.index[0], 'yardage_diff_2_3'] = 0

# Iterate through the DataFrame to calculate 'yardage_diff_1_2'
for index, row in lac_off_df.iterrows():
    if row['Down'] == 1:  # If Down is 1, set 'yardage_diff_1_2' to "yards to go"
        lac_off_df.at[index, 'yardage_diff_1_2'] = 0
    elif row['Down'] == 2:
        if index - 1 in lac_off_df.index:
            lac_off_df.at[index, 'yardage_diff_1_2'] = lac_off_df.at[index - 1, 'ToGo']-row['ToGo']
        else:
            lac_off_df.at[index, 'yardage_diff_1_2'] = 0
    elif row['Down'] == 3:
        lac_off_df.at[index, 'yardage_diff_1_2'] = 0
    elif row['Down'] == 4:  # If Down is 4, set 'yardage_diff_1_2' to 0
        lac_off_df.at[index, 'yardage_diff_1_2'] = 0


In [420]:
# Initialize the yardage difference column with zeros
lac_off_df['yardage_diff_2_3'] = 0

# Iterate over rows starting from the third row
for index, row in lac_off_df.iterrows():
    if index > 2 and row['Down'] == 3:
        previous_index = index - 1
        if previous_index in lac_off_df.index:  # Check if the previous index exists
            previous_row = lac_off_df.loc[previous_index]
            if previous_row['Down'] == 2:  # Ensure previous row's Down is 2
                lac_off_df.at[index, 'yardage_diff_2_3'] = previous_row['ToGo'] - row['ToGo']


In [421]:
lac_off_df['Quarter'] = lac_off_df['Quarter'].replace('OT', int('5'))
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,rush_play,rush_length,rush_yards_tally,pass_play,pass_complete,pass_incomplete,pass_length,pass_yards_tally,yardage_diff_1_2,yardage_diff_2_3
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,1.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,8.0,1.0,1.0,0.0,0.0,0.0,8.0,0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,1.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,1.0,2.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,...,0.0,0.0,12.0,1.0,1.0,0.0,9.0,9.0,2.0,0


In [422]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bintercepted\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'intercepted'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'intercepted'] = 0
lac_off_df['intercepted'] 
intercepton_count = (lac_off_df['intercepted'] > 0).sum()
print(f"interception count: {intercepton_count}")
#lac_off_df.head()

interception count: 12


In [423]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bfumbles\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'fumble'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'fumble'] = 0
fumble_count = (lac_off_df['fumble'] > 0).sum()
print(f"fumble count: {fumble_count}")

fumble count: 16


In [424]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bpunts\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'punt'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'punt'] = 0
punt_count = (lac_off_df['punt'] > 0).sum()
print(f"punt count: {punt_count}")
#lac_off_df.head()

punt count: 0


In [425]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bfield goal\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'field goal att'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'field goal att'] = 0
field_goal_count = (lac_off_df['field goal att'] > 0).sum()
print(f"field goal count: {field_goal_count}")
#lac_off_df.head()

field goal count: 0


In [426]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if 'sacked' in play_detail:  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'sacked'] = 1  # Set 'sacked' to 1 if 'sacked' is found
        # Extract sack yards using regex
        sack_yards_match = re.search(r'(-?\d+)\s+yards', play_detail)  # Match negative or positive numbers
        if sack_yards_match:
            sack_yards = int(sack_yards_match.group(1))  # Extract the sack yards value
            lac_off_df.at[index, 'sack_yards'] = sack_yards  # Assign sack yards to the corresponding row
        else:
            lac_off_df.at[index, 'sack_yards'] = 0  # If no sack yards found, set it to 0
    else:
        lac_off_df.at[index, 'sacked'] = 0
        lac_off_df.at[index, 'sack_yards'] = 0


for index, row in lac_off_df.iterrows():
    if index == 0:  
        lac_off_df.at[index, 'time_between_plays'] = 0  # or NaN
    elif row['Down'] > 1:
        if index - 1 in lac_off_df.index:
            lac_off_df.at[index, 'yardage_diff'] = abs(row['ToGo'] - lac_off_df.at[index - 1, 'ToGo'])
        else:
            lac_off_df.at[index, 'yardage_diff'] = 0  # or NaN
    elif row['Down'] == 1:
        lac_off_df.at[index, 'yardage_diff'] = row['ToGo']
lac_off_df.head()

In [427]:
def yards_to_promised_land(location):
    if isinstance(location, str):  # Check if location is a string
        location_parts = location.split()
        if len(location_parts) >= 2:  # Ensure there are at least two parts
            if "LAC" in location:  # Check if "LAC" is present in the location
                return 100-int(location_parts[1])   # 100-  yards if "LAC" is present
            else:
                return int(location_parts[1])  # Otherwise, return the yardage as is
    return None  # Return None for non-string values or invalid strings

# Apply the function to create the new column
lac_off_df['yards_to_endzone'] = lac_off_df['Location'].apply(yards_to_promised_land)

# Display the modified DataFrame
lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,pass_yards_tally,yardage_diff_1_2,yardage_diff_2_3,intercepted,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,75
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0,67
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,66
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,64
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,...,9.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,62


create new column "conversion_successful" if  while on third down the resultmof the play is a rush length or pass length is greater than 'yards to go', then put 1 in the conversion seuccesfful 

In [428]:
def update_1d_conversion_successful(row):
    if row['Down'] == 1:
        gain = row['rush_length'] + row['pass_length']  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['1D_conv_good'] = lac_off_df.apply(update_1d_conversion_successful, axis=1)
conv_1D_good_count = (lac_off_df['1D_conv_good']).sum()
print(conv_1D_good_count)
lac_off_df.head()


97


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,yardage_diff_1_2,yardage_diff_2_3,intercepted,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,75,0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0,67,0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,66,0
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,64,0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,...,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,62,0


In [429]:
def update_2d_conversion_successful(row):
    if row['Down'] == 2:
        gain = row['rush_length'] + row['pass_length']  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['2D_conv_good'] = lac_off_df.apply(update_2d_conversion_successful, axis=1)
conv_2D_good_count = (lac_off_df['2D_conv_good']).sum()
print(conv_2D_good_count)
lac_off_df.head()

108


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,yardage_diff_2_3,intercepted,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good,2D_conv_good
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0,0.0,0.0,0.0,0.0,0.0,0.0,75,0,0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0,0.0,0.0,0.0,0.0,0.0,0.0,67,0,0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0,0.0,0.0,0.0,0.0,0.0,0.0,66,0,0
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0,0.0,0.0,0.0,0.0,0.0,0.0,64,0,0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,...,0,0.0,0.0,0.0,0.0,0.0,0.0,62,0,1


In [430]:
def update_3d_conversion_successful(row):
    if row['Down'] == 3:
        gain = row['rush_length'] + row['pass_length']  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['3D_conv_good'] = lac_off_df.apply(update_3d_conversion_successful, axis=1)
conv_3D_good_count = (lac_off_df['3D_conv_good']).sum()
print(conv_3D_good_count)
lac_off_df.head()

113


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,intercepted,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0.0,0.0,0.0,0.0,75,0,0,0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,0.0,0.0,0.0,0.0,67,0,0,0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0.0,0.0,0.0,0.0,0.0,66,0,0,1
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,0,0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,...,0.0,0.0,0.0,0.0,0.0,0.0,62,0,1,0


In [431]:
def update_4d_conversion_successful(row):
    if row['Down'] == 4:
        gain = row['rush_length'] + row['pass_length']  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['4D_conv_good'] = lac_off_df.apply(update_4d_conversion_successful, axis=1)
conv_4D_good_count = (lac_off_df['4D_conv_good']).sum()
print(conv_4D_good_count)
lac_off_df.head()

13


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good,4D_conv_good
0,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0.0,0.0,0.0,75,0,0,0,0
1,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,0.0,0.0,0.0,67,0,0,0,0
2,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0.0,0.0,0.0,0.0,66,0,0,1,0
3,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0.0,0.0,0.0,64,0,0,0,0
4,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,...,0.0,0.0,0.0,0.0,0.0,62,0,1,0,0


In [432]:
lac_off_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232 entries, 0 to 1242
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   week              1232 non-null   int64  
 1   Quarter           1232 non-null   object 
 2   Time              1232 non-null   object 
 3   Down              1232 non-null   float64
 4   ToGo              1232 non-null   float64
 5   Location          1232 non-null   object 
 6   OPP               1232 non-null   object 
 7   OPP Score         1232 non-null   float64
 8   LAC Score         1232 non-null   float64
 9   Detail            1232 non-null   object 
 10  possession_team   1232 non-null   object 
 11  team change       1232 non-null   int64  
 12  rush_play         1232 non-null   float64
 13  rush_length       1232 non-null   float64
 14  rush_yards_tally  1232 non-null   float64
 15  pass_play         1232 non-null   float64
 16  pass_complete     1232 non-null   float64


In [433]:
lac_off_df.columns

Index(['week', 'Quarter', 'Time', 'Down', 'ToGo', 'Location', 'OPP',
       'OPP Score', 'LAC Score', 'Detail', 'possession_team', 'team change',
       'rush_play', 'rush_length', 'rush_yards_tally', 'pass_play',
       'pass_complete', 'pass_incomplete', 'pass_length', 'pass_yards_tally',
       'yardage_diff_1_2', 'yardage_diff_2_3', 'intercepted', 'fumble', 'punt',
       'field goal att', 'sacked', 'sack_yards', 'yards_to_endzone',
       '1D_conv_good', '2D_conv_good', '3D_conv_good', '4D_conv_good'],
      dtype='object')

In [434]:
column_order=['week','OPP','OPP Score', 'LAC Score','Time','Quarter','Location','yards_to_endzone','Detail','Down','ToGo','yardage_diff_1_2','yardage_diff_2_3','rush_length','pass_length','1D_conv_good','2D_conv_good', '3D_conv_good', '4D_conv_good','rush_play',  'rush_yards_tally','pass_play','pass_complete', 'pass_incomplete', 'pass_yards_tally','intercepted', 'fumble', 'punt','field goal att', 'sacked', 'sack_yards','possession_team', 'team change']

In [435]:
lac_off_df = lac_off_df.reindex(columns=column_order)
lac_off_df.head()

,week,OPP,OPP Score,LAC Score,Time,Quarter,Location,yards_to_endzone,Detail,Down,...,pass_incomplete,pass_yards_tally,intercepted,fumble,punt,field goal att,sacked,sack_yards,possession_team,team change
0,1,LVR,0.0,0.0,15:00,1,LAC 25,75,Austin Ekeler left tackle for 8 yards (tackle ...,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LAC,0
1,1,LVR,0.0,0.0,14:45,1,LAC 33,67,Justin Herbert pass complete short right to Ke...,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LAC,0
2,1,LVR,0.0,0.0,14:07,1,LAC 34,66,Zander Horvath up the middle for 2 yards (tack...,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LAC,0
3,1,LVR,0.0,0.0,13:24,1,LAC 36,64,Austin Ekeler right tackle for 2 yards (tackle...,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LAC,0
4,1,LVR,0.0,0.0,12:50,1,LAC 38,62,Justin Herbert pass complete short left to Jos...,2.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,LAC,0


In [436]:
lac_off_df.info()
#print("Sum of the values in the column:", column_sum)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232 entries, 0 to 1242
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   week              1232 non-null   int64  
 1   OPP               1232 non-null   object 
 2   OPP Score         1232 non-null   float64
 3   LAC Score         1232 non-null   float64
 4   Time              1232 non-null   object 
 5   Quarter           1232 non-null   object 
 6   Location          1232 non-null   object 
 7   yards_to_endzone  1232 non-null   int64  
 8   Detail            1232 non-null   object 
 9   Down              1232 non-null   float64
 10  ToGo              1232 non-null   float64
 11  yardage_diff_1_2  1232 non-null   float64
 12  yardage_diff_2_3  1232 non-null   int64  
 13  rush_length       1232 non-null   float64
 14  pass_length       1232 non-null   float64
 15  1D_conv_good      1232 non-null   int64  
 16  2D_conv_good      1232 non-null   int64  


In [437]:
lac_off_df = lac_off_df.astype({'LAC Score': 'int8', 'OPP Score': 'int8','Down':'int8','Down':'int8','Quarter': 'int8','OPP':'string','week':'int8','rush_length':'int8','yardage_diff_2_3':'int8','yardage_diff_1_2':'int8','ToGo':'int8','rush_length':'int8','pass_length':'int8','1D_conv_good':'int8','2D_conv_good':'int8','3D_conv_good':'int8','3D_conv_good':'int8','4D_conv_good':'int8','rush_play':'int8','rush_yards_tally':'int8','pass_play':'int8','pass_complete':'int8','pass_incomplete':'int8','pass_yards_tally':'int8','intercepted':'int8','fumble':'int8','punt':'int8','field goal att':'int8','sacked':'int8','sack_yards':'int8','team change':'int8','possession_team':'string','Location':'string','Detail':'string','yards_to_endzone':'int8'})
lac_off_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232 entries, 0 to 1242
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   week              1232 non-null   int8  
 1   OPP               1232 non-null   string
 2   OPP Score         1232 non-null   int8  
 3   LAC Score         1232 non-null   int8  
 4   Time              1232 non-null   object
 5   Quarter           1232 non-null   int8  
 6   Location          1232 non-null   string
 7   yards_to_endzone  1232 non-null   int8  
 8   Detail            1232 non-null   string
 9   Down              1232 non-null   int8  
 10  ToGo              1232 non-null   int8  
 11  yardage_diff_1_2  1232 non-null   int8  
 12  yardage_diff_2_3  1232 non-null   int8  
 13  rush_length       1232 non-null   int8  
 14  pass_length       1232 non-null   int8  
 15  1D_conv_good      1232 non-null   int8  
 16  2D_conv_good      1232 non-null   int8  
 17  3D_conv_good  

In [438]:
lac_off_clean_df=lac_off_df.copy()
lac_off_clean_df.head()

,week,OPP,OPP Score,LAC Score,Time,Quarter,Location,yards_to_endzone,Detail,Down,...,pass_incomplete,pass_yards_tally,intercepted,fumble,punt,field goal att,sacked,sack_yards,possession_team,team change
0,1,LVR,0,0,15:00,1,LAC 25,75,Austin Ekeler left tackle for 8 yards (tackle ...,1,...,0,0,0,0,0,0,0,0,LAC,0
1,1,LVR,0,0,14:45,1,LAC 33,67,Justin Herbert pass complete short right to Ke...,2,...,0,0,0,0,0,0,0,0,LAC,0
2,1,LVR,0,0,14:07,1,LAC 34,66,Zander Horvath up the middle for 2 yards (tack...,3,...,0,0,0,0,0,0,0,0,LAC,0
3,1,LVR,0,0,13:24,1,LAC 36,64,Austin Ekeler right tackle for 2 yards (tackle...,1,...,0,0,0,0,0,0,0,0,LAC,0
4,1,LVR,0,0,12:50,1,LAC 38,62,Justin Herbert pass complete short left to Jos...,2,...,0,9,0,0,0,0,0,0,LAC,0


In [439]:
lac_off_clean_df.to_csv("../data/lac_2022_off_clean", index=False)